# Calculate velocity errors over stable surfaces (Stable Surface Error) for all velocity maps in a folder

Generates a table with all velocity map parameters, including the stable surface error.

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import datetime
import fiona
import rasterio as rio
from rasterio.mask import mask
from shapely.geometry import mapping

### Set paths to velocity maps, the stable surface masks (ssm), and a glacier shapefile (UTM projection)

In [3]:
# path to autorift velocity maps:
autorift_outp = '/Volumes/SURGE_DISK/SR_autorift_output/'

# set path to folder with the stable surface mask (should have been generated when running custom autoRIFT)
# contains files named ssm_#m.tif
refvpath = '/Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/SR/'
print(os.listdir(refvpath))

# read in glacier shapefile and grab outline as shapes
shp_path = '/Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/SR/SR_Box_UTM43.shp'
with fiona.open(shp_path) as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

['SR_polygon_WGS.prj', 'SR_polygon_UTM43.dbf', 'SR_Box_WGS_UTM_43.prj', '.DS_Store', 'LS_images', 'SR_Box_WGS.dbf', 'SR_Box_WGS.shp', 'SR_Box_WGS.cpg', 'SR_Box_WGS.geojson', 'SR_Box_UTM43.prj', 'SR_Box_WGS.shx', 'SR_Box_WGS.qmd', 'SR_Box_UTM43.qpj', 'SR_polygon_UTM43.cpg', 'SR_polygon_UTM43.shp', 'SR_polygon_UTM43.qmd', 'SR_polygon_UTM43.shx', 'SR_Box_WGS_UTM_43.shx', 'SR_polygon_WGS.cpg', 'SR_polygon_WGS.shp', 'SR_Box_WGS_UTM_43.shp', 'SR_polygon_WGS.qmd', 'SR_polygon_WGS.shx', 'SR_itslive_data', 'LS9_20230913_5m_clipped.tif', 'SR_Box_UTM43.dbf', 'SR_Box_WGS.prj', 'LS9_20230913_5m_clipped.tif.aux.xml', 'SR_Box_UTM43.shp', 'SR_Box_UTM43.cpg', 'SR_polygon_UTM43.qpj', 'SR_Box_UTM43.shx', 'Screen Shot 2023-11-02 at 11.53.32 AM.png', 'SR_Box_WGS_UTM_43.dbf', 'ssm_uncropped.tif', 'SR_polygon_UTM43.prj', 'SR_polygon_WGS.dbf', 'ssm.tif']


### Grab important information and Stable Surface Error (SSE) from each velocity map

In [4]:
# initialize lists to store info for each velocity map
ds1s = []; ds2s = []; dts = []; chipsizes = []; sats = []; data_fracs = []; err_vx = []; err_vy = []; err_v = []; 
file_count = 0

# for each velocity map
for output in os.listdir(autorift_outp):
    if output.startswith('velocity') and output.endswith('.tif'):
        print(output) # display file names
        [filetype, ds1, ds2, chipsize, sat] = output.split('_') # grab info from filename
        d1 = datetime.datetime.strptime(ds1, '%Y%m%d'); d2 = datetime.datetime.strptime(ds2, '%Y%m%d')
        dt = d2-d1
        sat = sat.split('.')[0] # s
        ds1s.append(ds1); ds2s.append(ds2); dts.append(dt.days)
        chipsizes.append(int(chipsize[:-1])); sats.append(sat)
        
        # read in the velocity map
        v_reader = rio.open(autorift_outp+output)
        vx = v_reader.read(1); vy = v_reader.read(2) # grab vx and vy
        v = np.sqrt(vx**2 + vy**2) # calculate vmag from vx and vy
        # raster grid value (UTM coordinates)
        x_vals = np.arange(v_reader.bounds.left, v_reader.bounds.right+1, v_reader.transform[0])
        y_vals = np.arange(v_reader.bounds.bottom, v_reader.bounds.top+1, v_reader.transform[0])
        
        # Read in stable surface mask
        ssm_reader = rio.open(refvpath+'ssm_'+chipsize+'.tif') # stable surface mask
        ssm = ssm_reader.read(1)
        if ssm.shape != vx.shape:
            ssm = ssm[:vx.shape[0],:vx.shape[1]]
        
        # grab data fraction in glacier outline
        cropped_img, cropped_transform = mask(v_reader, shapes, nodata= -3e5, crop=True)
        if cropped_img.shape[0] == 3:
            cropped_v = cropped_img[2] # grab the third band [2]
        else:
            cropped_vx = cropped_img[0]; cropped_vy = cropped_img[1]
            cropped_v = np.sqrt(cropped_vx**2 + cropped_vy**2) 
        cropped_v_filled = cropped_v.copy()
        cropped_v_filled[np.isnan(cropped_v_filled)] = 0 # fill all Nans with 0
        total_pixels = np.count_nonzero(cropped_v_filled >= 0) # count number of non-nodata values (negative)
        cropped_v[cropped_v < 0] = np.NaN # turn no data values (negative) into NaNs
        pixels_w_data = len(cropped_v[cropped_v >= 0]) # count non Nans (number of pixels with data)
        if pixels_w_data > 0:
            data_percent = int(pixels_w_data/total_pixels*100)
            data_fracs.append(data_percent)
            empty = False
        else:
            data_fracs.append(0)
            empty = True
#         plt.imshow(cropped_v); plt.colorbar(); plt.title(str(data_percent)+' %'); plt.show() # check visually
            
        # Grab off-ice velocities using stable surface mask
        ssm[ssm > 0] = 2 # turn ice into 2
        ssm[ssm == 0] = 1 # turn stable surfaces into 1
        ssm[ssm == 2] = 0 # turn ice into 0s
        ssm_masked_vx = ssm*vx; ssm_masked_vy = ssm*vy; ssm_masked_v = ssm*v 
        
        # calculate RMSE [m/d]
        err_vx.append(np.sqrt(np.nanmean(ssm_masked_vx**2))/365)
        err_vy.append(np.sqrt(np.nanmean(ssm_masked_vy**2))/365)
        err_v.append(np.sqrt(np.nanmean(ssm_masked_v**2))/365)
        
        # get ready for next iteration
        del v, vx, vy, empty
        file_count += 1
        
print(file_count, 'files found.')

velocity_20201130_20201218_200m_S2.tif


RasterioIOError: /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/SR/ssm_200m.tif: No such file or directory

In [23]:
df = pd.DataFrame(list(zip(ds1s, ds2s, chipsizes, sats, dts, data_fracs, err_vx, err_vy, err_v)),
                 columns=['ds1','ds2','min_chip_size','sat','dt_days','data_percent',
                          'err_vx','err_vy','err_v'])
df.sort_values(by=['ds1','min_chip_size','sat'])
df

,ds1,ds2,min_chip_size,sat,dt_days,data_percent,err_vx,err_vy,err_v
0,20211010,20211016,100,PS,6,88,0.793093,0.916111,1.211716
1,20210602,20210608,200,PS,6,52,0.529784,0.434593,0.685231
2,20200904,20200911,200,PS,7,23,0.500644,0.355475,0.614009
3,20211016,20211021,200,PS,5,82,1.430727,0.717657,1.600629
4,20210402,20210407,100,PS,5,10,0.899796,2.663131,2.811032
...,...,...,...,...,...,...,...,...,...
98,20200213,20200224,100,PS,11,49,0.258656,0.217356,0.337856
99,20200224,20200306,200,PS,11,51,0.269099,0.170366,0.318494
100,20210716,20210730,100,PS,14,63,0.153246,0.071999,0.169317
101,20210730,20210804,100,PS,5,67,0.302962,0.498794,0.583594


In [26]:
# convert dates to datetime objects
df.ds1 = pd.to_datetime(df.ds1, format='%Y%m%d')
df.ds2 = pd.to_datetime(df.ds2, format='%Y%m%d')
df['mid_date'] = (df.ds2-df.ds1)/2+df.ds1
df = df[df.min_chip_size == 100]
df = df.dropna()
df

/Users/jukesliu/opt/anaconda3/envs/newautoriftenv/lib/python3.9/site-packages/pandas/core/generic.py:5507: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/jukesliu/opt/anaconda3/envs/newautoriftenv/lib/python3.9/site-packages/pandas/core/generic.py:5507: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/var/folders/mr/7f69f7y50zj85n420pyvh8cr0000gq/T/ipykernel_92874/3306725152.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

,ds1,ds2,min_chip_size,sat,dt_days,data_percent,err_vx,err_vy,err_v,mid_date
0,2021-10-10,2021-10-16,100,PS,6,88,0.793093,0.916111,1.211716,2021-10-13 00:00:00
4,2021-04-02,2021-04-07,100,PS,5,10,0.899796,2.663131,2.811032,2021-04-04 12:00:00
6,2020-03-22,2020-03-29,100,PS,7,79,0.370300,0.486965,0.611765,2020-03-25 12:00:00
7,2020-03-29,2020-04-04,100,PS,6,74,0.651614,0.762402,1.002925,2020-04-01 00:00:00
11,2021-05-25,2021-06-02,100,PS,8,19,0.795669,1.184228,1.426705,2021-05-29 00:00:00
12,2021-03-28,2021-04-02,100,PS,5,43,0.989818,1.210673,1.563799,2021-03-30 12:00:00
14,2021-03-03,2021-03-08,100,PS,5,59,0.623016,0.343979,0.711667,2021-03-05 12:00:00
18,2021-09-07,2021-09-18,100,PS,11,51,0.369285,0.434977,0.570592,2021-09-12 12:00:00
19,2020-05-27,2020-07-01,100,PS,35,46,0.089555,0.053689,0.104416,2020-06-13 12:00:00
20,2021-07-05,2021-07-16,100,PS,11,74,0.149519,0.087862,0.173423,2021-07-10 12:00:00


### Export CSV to a file

In [27]:
outfilepath = '/Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/error_csvs/PS_dt5to60.csv' # enter path and name of CSV file to save to
df.to_csv(outfilepath)

## Plot errors

In [ ]:
# create subplots with common y-axis
fs = 12
a = 0.3 # control opacity
colors_dict = {'LS':'tab:blue', 'S2':'tab:orange', 'S1':'tab:green', 'PS':'tab:red'}
fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(14,8), sharey=True)

# 1) plot velocity residuals over stable surfaces
ax1.set_title("Velocity over stable surfaces", fontsize=fs)
ax1.plot(df.err_vx,df.mid_date, 'kx', alpha=a)
ax1.plot(df.err_vy,df.mid_date, 'gx', alpha=a)
ax1.set_xlim(0, 4)
ax1.set_xlabel('RMSE [m/d]', fontsize=fs); ax1.set_ylabel('Velocity map mid date', fontsize=fs);
ax1.legend(['X error','Y error'], loc='lower right', fontsize=fs)

# 2) plot speed residual of stable surfaces (scaled by max speed)
ax2.set_title("Speed error", fontsize=fs)
ax2.scatter(df.err_v, df.mid_date, c=df.sat.map(colors_dict), alpha=a) # color points by satellite (see colors dict)
ax2.set_xlim(0,4)
ax2.set_xlabel('RMSE [m/d]', fontsize=fs);

# 3) percent coverage on glacier
ax3.set_title("Data coverage")
ax3.hlines(df.mid_date, np.zeros(len(df.data_percent)),df.data_percent, color='k',alpha=a)
ax3.set_xlabel('Pixels with data on glacier [%]', fontsize=fs)
ax3.set_xlim(0,100)

# all plots
for ax in [ax1, ax2, ax3]:
    ax.tick_params(labelsize=fs)
    ax.grid()
    ax.set_ylim(datetime.datetime(2013,1,1), datetime.datetime(2022,1,1))

plt.suptitle('Summary of velocity map errors and coverage', fontsize=fs+4)
plt.show()